# Crear Mosaico Imagenes google Earth Engine 

In [1]:
# Importar GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar shp cuenca
cuenca = ee.FeatureCollection("users/bravomoralesnino/SHP/Cuenca")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap = 'SATELLITE')

In [4]:
geometria = cuenca.geometry()
Map.centerObject(geometria, 8)

In [5]:
Map.addLayer(cuenca, {"color" : "00FF11"},name = "Cuenca")
Map

Map(center=[-13.822942378556581, -71.5432582449152], controls=(WidgetControl(options=['position', 'transparent…

## Seleccion colleccion Landsat 8 RS

In [6]:
# Coleccion de Landsat 8 OLI TIRS RS
L8_RS = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
            .filterDate('2019-01-01','2019-12-31')\
            .filterMetadata('CLOUD_COVER','less_than',10)\
            .filterBounds(cuenca)

In [7]:
# Conteo de imagenes
count = L8_RS.size()
print("Cantidad de imagenes L8_RS:", count.getInfo())

Cantidad de imagenes L8_RS: 16


In [9]:
# Imprimir la lista de ID Imagenes
catalogo_L8 = L8_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(catalogo_L8)

['LC08_003069_20190808', 'LC08_003069_20190925', 'LC08_003070_20190520', 'LC08_003070_20190605', 'LC08_003070_20190621', 'LC08_003070_20190707', 'LC08_003070_20190808', 'LC08_003070_20190925', 'LC08_003070_20191011', 'LC08_004069_20190527', 'LC08_004070_20190527', 'LC08_004070_20190612', 'LC08_004070_20190714', 'LC08_004070_20190730', 'LC08_004070_20190815', 'LC08_004070_20190831']


In [10]:
type(catalogo_L8)

list

In [11]:
catalogo_L8[9]

'LC08_004069_20190527'

## Seleccion imagen Landsat 8

In [13]:
# Importar imagen Landsat 8 RS 
image1 = ee.Image("LANDSAT/LC08/C01/T1_SR" + "/" + catalogo_L8[0]) # 3/69
image2 = ee.Image("LANDSAT/LC08/C01/T1_SR" + "/" + catalogo_L8[2]) # 3/70
image3 = ee.Image("LANDSAT/LC08/C01/T1_SR" + "/" + catalogo_L8[9]) # 4/69
image4 = ee.Image("LANDSAT/LC08/C01/T1_SR" + "/" + catalogo_L8[10]) # 4/70

In [14]:
# Nombre de las bandas
print(image1.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


In [15]:
# Simbologia estala 0 - 10000
viz_RS = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 250,
    'max': 7000,
    'gamma': 1.90
}

In [17]:
# Visualizar ESCALADO 0 - 10000
Map.addLayer(image1, viz_RS,name = "image1")
Map.addLayer(image2, viz_RS,name = "image2")
Map.addLayer(image3, viz_RS,name = "image3")
Map.addLayer(image4, viz_RS,name = "image4")
Map.addLayer(cuenca, {"color" : "00FF11"},name = "Cuenca")
Map

Map(bottom=36040.0, center=[-16.109153239219467, -71.62261962890626], controls=(WidgetControl(options=['positi…

## Realizar una collección de imagenes

In [18]:
# Seleccionar varias imagenes
selec_imagenes = ee.ImageCollection.fromImages([image1,image2,image3,image4])

In [19]:
type(selec_imagenes)

ee.imagecollection.ImageCollection

In [20]:
# crear mosaico
mosaico = selec_imagenes.mosaic()

In [21]:
# Visualizar ESCALADO 0 - 10000
Map.addLayer(mosaico, viz_RS,name = "mosaico")
Map

Map(bottom=17970.0, center=[-13.987376214146455, -70.47180175781251], controls=(WidgetControl(options=['positi…

## Seleccion de bandas

In [22]:
# Consultar nombre bandas
print(mosaico.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']


## Crear mosaico

In [23]:
mosaico = mosaico.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])

## Factor de escala

In [24]:
# Escalar la imagen Landsat 8 RS 0 - 1
mosaico_Es = mosaico.multiply(0.0001)

In [25]:
# Consultar Proyeccion
mosaico_Es.projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

## Reproyección zona

In [26]:
# Reproyectar segun zona de estudio
mosaico_Es_UTM = mosaico_Es.reproject(crs="EPSG:32719", scale = 30)

In [27]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

## Recortar imagen zona

In [28]:
# Recortar del area de estudio
mosaico_clip = mosaico_Es_UTM.clip(cuenca)

In [29]:
# Visualizar ESCALADO 0 - 1
Map.addLayer(mosaico_clip, viz_es,name = "mosaico clip")
Map

Map(bottom=17970.0, center=[-13.987376214146455, -70.47180175781251], controls=(WidgetControl(options=['positi…